<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Chains Exercise 

## TASK:
Fill out the function below that takes in a string input Customer Support email that could be written in any language. The function will then detect the language, translate the email, and provide a summary.

Fill out the function below using a Sequential Chain, the function should do the following:

1. Detect the language the email is written in
2. Translate the email from detected language to English
3. Return a summary of the translated email

Note: The Function should return a dictionary that contains all three of these outputs!

## Reading in our example Email Text File:

In [1]:
spanish_email = open('spanish_customer_email.txt').read()

In [2]:
print(spanish_email)

Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAAS

Estimado Equipo de Soporte al Cliente,

Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS. Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.

Me gustaría describir detalladamente el problema que estoy experimentando:

1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.

2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica q

## Function to fill out:

In [41]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables.base import RunnableLambda
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_core.prompts.chat import AIMessagePromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate

In [75]:
def translate_and_summarize(email):
    """
    Translates an email written in a detected language to English and generates a summary.

    Args:
        email (str): The email to be processed and translated.

    Returns:
        dict: A dictionary containing the following keys:
            - 'language': The language the email was written in.
            - 'translated_email': The translated version of the email in English.
            - 'summary': A short summary of the translated email.

    Raises:
        Exception: If any error occurs during the LLM chain execution.

    Example:
        email = "Hola, ¿cómo estás? Espero que todo vaya bien."
        result = translate_and_summarize(email)
        print(result)
        # Output:
        # {
        #     'language': 'Spanish',
        #     'translated_email': 'Hello, how are you? I hope everything is going well.',
        #     'summary': 'A friendly greeting and a wish for well-being.'
        # }
    """
    output = {}
    output['email'] = email
    chat= ChatOllama(model= 'llama3.2:1b', temperature= 0)
        # Step 1: Detect language
    detect_prompt = ChatPromptTemplate.from_template(
        "Identify the language of this email:\n\n{email}\n\nOnly respond with the language name in English."
    )
    detect_chain = detect_prompt | chat
    detect_result = detect_chain.invoke({"email": email})

    # Step 2: Clean up the language output
    langPrompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("Extract the language name from this sentence, return only a single word."),
        HumanMessagePromptTemplate.from_template("The language of this email is Spanish, specifically a variant of the Castilian Spanish dialect."),
        AIMessagePromptTemplate.from_template("Spanish"),
        HumanMessagePromptTemplate.from_template("{sentence}")
    ])
    lang_chain = langPrompt | chat

    # Clean up the result from the detection step
    language_result = lang_chain.invoke({"sentence": detect_result.content})
    language = language_result.content.strip()
    output["language"] = language

    # Step 3: Translate

    # translate_prompt = ChatPromptTemplate.from_template(
    #     "The email is written in {language} language, translate the given email into english language\n\n {email} ."
    # )
    translate_prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("You are a AI translator, An email is written in certain language you need translate it to another langauge"),
        HumanMessagePromptTemplate.from_template("The email is written in spanish language, translate the given email into english language\n\n Hola, ¿cómo estás? Espero que todo vaya bien. "),
        AIMessagePromptTemplate.from_template('Hello, how are you? I hope everything is going well.'),
        HumanMessagePromptTemplate.from_template("The email is written in {language} language, translate the given email into english language\n\n {email} .")
    ])

    translate_chain = translate_prompt | chat

    # Run final translation
    result = translate_chain.invoke({"language": language, "email": email})
    output["translated_email"] = result.content.split("English:\n\n")[-1]

    # Step 3: Summary

    summaryPrompt = ChatPromptTemplate.from_template(template="Give the summary of the given text\n\n {text}")
    summaryChain = summaryPrompt | chat
    output['summary'] = summaryChain.invoke({'text' : output["translated_email"]}).content

    return output

## Example Usage and Outputs

In [76]:
result = translate_and_summarize(spanish_email)

In [77]:
result.keys()

dict_keys(['email', 'language', 'translated_email', 'summary'])

In [81]:
print(result['email'])

Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAAS

Estimado Equipo de Soporte al Cliente,

Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS. Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.

Me gustaría describir detalladamente el problema que estoy experimentando:

1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.

2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica q

In [78]:
print(result['language'])

Castilian


In [79]:
print(result['translated_email'])

Subject: Technical Report - Panel SAAS Functionality

Dear Customer Support Team,

I hope this message finds you well. I am writing to inform you about a technical issue that I have encountered while using your SAAS product. As a loyal customer, I appreciate the value that your product brings to my business, but currently, I am facing an obstacle that requires your expertise.

I would like to describe in detail the problem I am experiencing:

1. Graphics Issue: When logging into the SAAS panel, I have noticed that the graphics and tables on the main page do not render correctly. The data points appear distorted, and some elements overlap, making it difficult to interpret the information accurately.

2. Export Functionality Issue: Additionally, I have been unable to export reports and data from the panel. Each time I try to export a report in CSV or PDF format, I receive an error message indicating that the export failed. This functionality is crucial for sharing information with my tea

In [80]:
print(result['summary'])

The customer, Maria Rodriguez, is experiencing several technical issues with the SAAS (Software as a Service) panel of their company's software product. Specifically, she is facing:

1. Graphics issues on the main page, where data points appear distorted and elements overlap.
2. Export functionality problems, which prevent her from exporting reports in CSV or PDF format.
3. Slow loading times, which affect her productivity.

Maria has tried clearing browser cache and cookies, testing the panel with different browsers, but these issues persist. She is requesting assistance from the customer support team to resolve these technical issues, including:

1. Correcting the graphics problems on the main page.
2. Resolving the export functionality issue.
3. Optimizing the performance of the SAAS panel.

Maria appreciates prompt attention and looks forward to receiving a response and resolving these technical issues as soon as possible.
